# Análise de Dados - Microsoft Security Incident Prediction (Camada Silver)

Este notebook realiza análises exploratórias e estatísticas dos dados processados da camada Silver do dataset Microsoft Security Incident Prediction.

## Objetivos da Análise
- Compreender a distribuição dos dados
- Identificar padrões e tendências
- Analisar a qualidade dos dados processados
- Preparar insights para modelagem de Machine Learning


## 1. Importação das Bibliotecas e Carregamento dos Dados


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações para visualização
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Carregamento dos dados processados da camada Silver
df = pd.read_csv('security_incident_prediction_silver.csv', low_memory=False)

print("Dataset carregado com sucesso!")
print(f"Dimensões do dataset: {df.shape}")
print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Visualizar as primeiras linhas
df.head()


## 2. Análise Geral dos Dados


In [ ]:
# Informações gerais sobre o dataset
print("=== INFORMAÇÕES GERAIS DO DATASET ===")
print(f"Número de registros: {len(df):,}")
print(f"Número de colunas: {len(df.columns)}")
print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n=== TIPOS DE DADOS ===")
print(df.dtypes.value_counts())

print("\n=== INFORMAÇÕES DETALHADAS ===")
df.info()

print("\n=== ESTATÍSTICAS DESCRITIVAS ===")
df.describe()


## 3. Análise da Variável Target (IncidentGrade)


In [ ]:
# Análise da variável target IncidentGrade
if 'incidentgrade' in df.columns:
    target_counts = df['incidentgrade'].value_counts()
    target_percentages = df['incidentgrade'].value_counts(normalize=True) * 100
    
    print("=== DISTRIBUIÇÃO DA VARIÁVEL TARGET (IncidentGrade) ===")
    print("\nContagem por classe:")
    for value, count in target_counts.items():
        print(f"  Classe {value}: {count:,} ({target_percentages[value]:.2f}%)")
    
    # Mapear os valores para nomes mais legíveis (baseado no dataset original)
    class_mapping = {0: 'FalsePositive', 1: 'TruePositive', 2: 'BenignPositive'}
    print("\nMapeamento das classes:")
    for encoded_val, original_val in class_mapping.items():
        if encoded_val in target_counts.index:
            count = target_counts[encoded_val]
            percentage = target_percentages[encoded_val]
            print(f"  {encoded_val} = {original_val}: {count:,} ({percentage:.2f}%)")
    
    # Visualização da distribuição
    plt.figure(figsize=(12, 6))
    
    # Subplot 1: Gráfico de barras
    plt.subplot(1, 2, 1)
    bars = plt.bar(target_counts.index, target_counts.values, color=['#ff7f7f', '#7fbf7f', '#7f7fff'])
    plt.title('Distribuição da Variável Target (IncidentGrade)')
    plt.xlabel('Classe')
    plt.ylabel('Contagem')
    
    # Adicionar percentuais nas barras
    for i, (bar, count, pct) in enumerate(zip(bars, target_counts.values, target_percentages.values)):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + bar.get_height()*0.01,
                f'{count:,}\n({pct:.1f}%)', ha='center', va='bottom')
    
    # Subplot 2: Gráfico de pizza
    plt.subplot(1, 2, 2)
    labels = [f'{class_mapping.get(i, f"Classe {i}")}\n({pct:.1f}%)' 
              for i, pct in zip(target_counts.index, target_percentages.values)]
    plt.pie(target_counts.values, labels=labels, autopct='%1.1f%%', startangle=90)
    plt.title('Distribuição Percentual das Classes')
    
    plt.tight_layout()
    plt.show()
    
    # Análise de balanceamento
    balance_ratio = min(target_counts) / max(target_counts)
    print(f"\nRazão de balanceamento: {balance_ratio:.3f}")
    if balance_ratio < 0.1:
        print("  Dataset altamente desbalanceado!")
    elif balance_ratio < 0.3:
        print("  Dataset moderadamente desbalanceado")
    else:
        print(" Dataset relativamente balanceado")
        
else:
    print("Variável 'incidentgrade' não encontrada no dataset.")
    print("Colunas disponíveis:", df.columns.tolist())


## 4. Análise de Categorias de Incidentes


In [ ]:
# Análise das categorias de incidentes
if 'category' in df.columns:
    category_counts = df['category'].value_counts()
    category_percentages = df['category'].value_counts(normalize=True) * 100
    
    print("=== ANÁLISE DE CATEGORIAS DE INCIDENTES ===")
    print(f"Total de categorias únicas: {len(category_counts)}")
    
    print("\nTop 10 categorias mais frequentes:")
    for i, (category, count) in enumerate(category_counts.head(10).items()):
        percentage = category_percentages[category]
        print(f"  {i+1:2d}. Categoria {category}: {count:,} ({percentage:.2f}%)")
    
    # Visualização das categorias
    plt.figure(figsize=(15, 8))
    
    # Gráfico de barras horizontais para as top categorias
    top_categories = category_counts.head(15)
    plt.barh(range(len(top_categories)), top_categories.values, color='skyblue')
    plt.yticks(range(len(top_categories)), [f'Categoria {cat}' for cat in top_categories.index])
    plt.xlabel('Número de Incidentes')
    plt.title('Top 15 Categorias de Incidentes por Frequência')
    plt.gca().invert_yaxis()
    
    # Adicionar valores nas barras
    for i, (cat, count) in enumerate(top_categories.items()):
        plt.text(count + count*0.01, i, f'{count:,}', va='center', ha='left')
    
    plt.tight_layout()
    plt.show()
    
    # Análise de distribuição por categoria e incident grade
    if 'incidentgrade' in df.columns:
        print("\n=== DISTRIBUIÇÃO DE CATEGORIAS POR TIPO DE INCIDENTE ===")
        cross_tab = pd.crosstab(df['category'], df['incidentgrade'], normalize='index') * 100
        
        # Mostrar as top 5 categorias
        top_5_categories = category_counts.head(5).index
        print("\nDistribuição das Top 5 categorias por tipo de incidente:")
        for cat in top_5_categories:
            if cat in cross_tab.index:
                print(f"\nCategoria {cat}:")
                for grade in cross_tab.columns:
                    percentage = cross_tab.loc[cat, grade]
                    grade_name = {0: 'FalsePositive', 1: 'TruePositive', 2: 'BenignPositive'}.get(grade, f'Grade{grade}')
                    print(f"  {grade_name}: {percentage:.2f}%")
    
else:
    print("Coluna 'category' não encontrada no dataset.")


## 5. Análise Temporal dos Incidentes


In [ ]:
# Análise temporal dos incidentes
if 'timestamp' in df.columns:
    # Converter timestamp para datetime
    df['datetime'] = pd.to_datetime(df['timestamp'], errors='coerce')
    
    # Extrair componentes temporais
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['day_name'] = df['datetime'].dt.day_name()
    
    print("=== ANÁLISE TEMPORAL DOS INCIDENTES ===")
    
    # Período dos dados
    print(f"Período dos dados:")
    print(f"  Data inicial: {df['datetime'].min()}")
    print(f"  Data final: {df['datetime'].max()}")
    print(f"  Total de dias: {(df['datetime'].max() - df['datetime'].min()).days}")
    
    # Análise por ano
    yearly_counts = df['year'].value_counts().sort_index()
    print(f"\n=== INCIDENTES POR ANO ===")
    for year, count in yearly_counts.items():
        print(f"  {year}: {count:,} incidentes")
    
    # Análise por mês
    monthly_counts = df['month'].value_counts().sort_index()
    print(f"\n=== INCIDENTES POR MÊS ===")
    month_names = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 
                   'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
    for month, count in monthly_counts.items():
        print(f"  {month_names[month-1]} ({month:02d}): {count:,} incidentes")
    
    # Análise por dia da semana
    daily_counts = df['day_of_week'].value_counts().sort_index()
    print(f"\n=== INCIDENTES POR DIA DA SEMANA ===")
    day_names = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo']
    for day, count in daily_counts.items():
        print(f"  {day_names[day]}: {count:,} incidentes")
    
    # Análise por hora
    hourly_counts = df['hour'].value_counts().sort_index()
    print(f"\n=== INCIDENTES POR HORA DO DIA ===")
    print("Top 5 horas com mais incidentes:")
    for hour, count in hourly_counts.head(5).items():
        print(f"  {hour:02d}:00 - {hour:02d}:59: {count:,} incidentes")
    
    # Visualizações temporais
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Gráfico 1: Incidentes por mês
    axes[0, 0].bar(monthly_counts.index, monthly_counts.values, color='lightblue')
    axes[0, 0].set_title('Incidentes por Mês')
    axes[0, 0].set_xlabel('Mês')
    axes[0, 0].set_ylabel('Número de Incidentes')
    axes[0, 0].set_xticks(monthly_counts.index)
    axes[0, 0].set_xticklabels([month_names[i-1] for i in monthly_counts.index], rotation=45)
    
    # Gráfico 2: Incidentes por dia da semana
    axes[0, 1].bar(daily_counts.index, daily_counts.values, color='lightgreen')
    axes[0, 1].set_title('Incidentes por Dia da Semana')
    axes[0, 1].set_xlabel('Dia da Semana')
    axes[0, 1].set_ylabel('Número de Incidentes')
    axes[0, 1].set_xticks(daily_counts.index)
    axes[0, 1].set_xticklabels([day_names[i] for i in daily_counts.index], rotation=45)
    
    # Gráfico 3: Incidentes por hora
    axes[1, 0].plot(hourly_counts.index, hourly_counts.values, marker='o', color='red')
    axes[1, 0].set_title('Incidentes por Hora do Dia')
    axes[1, 0].set_xlabel('Hora')
    axes[1, 0].set_ylabel('Número de Incidentes')
    axes[1, 0].set_xticks(range(0, 24, 2))
    axes[1, 0].grid(True, alpha=0.3)
    
    # Gráfico 4: Timeline de incidentes
    daily_incidents = df.groupby(df['datetime'].dt.date).size()
    axes[1, 1].plot(daily_incidents.index, daily_incidents.values, alpha=0.7, color='purple')
    axes[1, 1].set_title('Timeline de Incidentes (Diário)')
    axes[1, 1].set_xlabel('Data')
    axes[1, 1].set_ylabel('Número de Incidentes')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Coluna 'timestamp' não encontrada no dataset.")


## 6. Análise de Correlações e Padrões


In [ ]:
# Análise de correlações e padrões
print("=== ANÁLISE DE CORRELAÇÕES ===")

# Selecionar colunas numéricas para análise de correlação
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Colunas numéricas disponíveis: {len(numeric_columns)}")

if len(numeric_columns) > 1:
    # Calcular matriz de correlação
    correlation_matrix = df[numeric_columns].corr()
    
    # Encontrar correlações mais altas (excluindo correlação com si mesmo)
    high_correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            col1 = correlation_matrix.columns[i]
            col2 = correlation_matrix.columns[j]
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > 0.5:  # Correlação significativa
                high_correlations.append((col1, col2, corr_value))
    
    # Ordenar por valor absoluto da correlação
    high_correlations.sort(key=lambda x: abs(x[2]), reverse=True)
    
    print(f"\nCorrelações significativas (|r| > 0.5):")
    if high_correlations:
        for col1, col2, corr in high_correlations[:10]:  # Top 10
            print(f"  {col1} ↔ {col2}: {corr:.3f}")
    else:
        print("  Nenhuma correlação significativa encontrada.")
    
    # Visualização da matriz de correlação
    plt.figure(figsize=(12, 10))
    
    # Criar heatmap de correlação
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Máscara para mostrar apenas metade
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.2f', cbar_kws={"shrink": .8})
    plt.title('Matriz de Correlação das Variáveis Numéricas')
    plt.tight_layout()
    plt.show()
    
    # Análise de correlação com a variável target
    if 'incidentgrade' in numeric_columns:
        target_correlations = correlation_matrix['incidentgrade'].drop('incidentgrade').sort_values(key=abs, ascending=False)
        
        print(f"\n=== CORRELAÇÕES COM A VARIÁVEL TARGET (IncidentGrade) ===")
        print("Top 10 variáveis mais correlacionadas com IncidentGrade:")
        for col, corr in target_correlations.head(10).items():
            print(f"  {col}: {corr:.3f}")
        
        # Visualização das correlações com target
        plt.figure(figsize=(10, 8))
        target_correlations_abs = target_correlations.abs()
        top_15 = target_correlations_abs.head(15)
        
        plt.barh(range(len(top_15)), top_15.values, color='lightcoral')
        plt.yticks(range(len(top_15)), top_15.index)
        plt.xlabel('Correlação Absoluta com IncidentGrade')
        plt.title('Top 15 Variáveis com Maior Correlação Absoluta com IncidentGrade')
        plt.gca().invert_yaxis()
        
        # Adicionar valores nas barras
        for i, (col, corr) in enumerate(zip(top_15.index, target_correlations[top_15.index])):
            plt.text(corr + 0.01 if corr >= 0 else corr - 0.01, i, f'{corr:.3f}', 
                    va='center', ha='left' if corr >= 0 else 'right')
        
        plt.tight_layout()
        plt.show()

else:
    print("Poucas colunas numéricas disponíveis para análise de correlação.")


## 7. Análise Geográfica dos Incidentes


In [ ]:
# Análise geográfica dos incidentes
print("=== ANÁLISE GEOGRÁFICA DOS INCIDENTES ===")

# Análise por país
if 'countrycode' in df.columns:
    country_counts = df['countrycode'].value_counts()
    print(f"Total de países únicos: {len(country_counts)}")
    
    print("\nTop 10 países com mais incidentes:")
    for i, (country, count) in enumerate(country_counts.head(10).items()):
        percentage = (count / len(df)) * 100
        print(f"  {i+1:2d}. País {country}: {count:,} ({percentage:.2f}%)")
    
    # Análise por estado (se disponível)
    if 'state' in df.columns:
        state_counts = df['state'].value_counts()
        print(f"\nTotal de estados únicos: {len(state_counts)}")
        
        print("\nTop 10 estados com mais incidentes:")
        for i, (state, count) in enumerate(state_counts.head(10).items()):
            percentage = (count / len(df)) * 100
            print(f"  {i+1:2d}. Estado {state}: {count:,} ({percentage:.2f}%)")
    
    # Análise por cidade (se disponível)
    if 'city' in df.columns:
        city_counts = df['city'].value_counts()
        print(f"\nTotal de cidades únicas: {len(city_counts)}")
        
        print("\nTop 10 cidades com mais incidentes:")
        for i, (city, count) in enumerate(city_counts.head(10).items()):
            percentage = (count / len(df)) * 100
            print(f"  {i+1:2d}. Cidade {city}: {count:,} ({percentage:.2f}%)")
    
    # Visualizações geográficas
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Gráfico 1: Top países
    top_countries = country_counts.head(10)
    axes[0, 0].barh(range(len(top_countries)), top_countries.values, color='lightblue')
    axes[0, 0].set_yticks(range(len(top_countries)))
    axes[0, 0].set_yticklabels([f'País {country}' for country in top_countries.index])
    axes[0, 0].set_xlabel('Número de Incidentes')
    axes[0, 0].set_title('Top 10 Países por Número de Incidentes')
    axes[0, 0].invert_yaxis()
    
    # Gráfico 2: Top estados (se disponível)
    if 'state' in df.columns:
        top_states = state_counts.head(10)
        axes[0, 1].barh(range(len(top_states)), top_states.values, color='lightgreen')
        axes[0, 1].set_yticks(range(len(top_states)))
        axes[0, 1].set_yticklabels([f'Estado {state}' for state in top_states.index])
        axes[0, 1].set_xlabel('Número de Incidentes')
        axes[0, 1].set_title('Top 10 Estados por Número de Incidentes')
        axes[0, 1].invert_yaxis()
    else:
        axes[0, 1].text(0.5, 0.5, 'Dados de estado não disponíveis', 
                        ha='center', va='center', transform=axes[0, 1].transAxes)
        axes[0, 1].set_title('Análise por Estado')
    
    # Gráfico 3: Distribuição geográfica por tipo de incidente
    if 'incidentgrade' in df.columns:
        geo_incident = pd.crosstab(df['countrycode'], df['incidentgrade'])
        top_5_countries = country_counts.head(5).index
        
        # Preparar dados para o gráfico
        incident_types = geo_incident.columns
        x = np.arange(len(top_5_countries))
        width = 0.25
        
        for i, incident_type in enumerate(incident_types):
            values = [geo_incident.loc[country, incident_type] if country in geo_incident.index else 0 
                     for country in top_5_countries]
            axes[1, 0].bar(x + i*width, values, width, 
                          label=f'Grade {incident_type}')
        
        axes[1, 0].set_xlabel('País')
        axes[1, 0].set_ylabel('Número de Incidentes')
        axes[1, 0].set_title('Distribuição de Tipos de Incidente por País')
        axes[1, 0].set_xticks(x + width)
        axes[1, 0].set_xticklabels([f'País {country}' for country in top_5_countries])
        axes[1, 0].legend()
    
    # Gráfico 4: Concentração de incidentes
    if 'city' in df.columns:
        top_cities = city_counts.head(10)
        axes[1, 1].bar(range(len(top_cities)), top_cities.values, color='orange')
        axes[1, 1].set_xticks(range(len(top_cities)))
        axes[1, 1].set_xticklabels([f'Cidade {city}' for city in top_cities.index], rotation=45)
        axes[1, 1].set_ylabel('Número de Incidentes')
        axes[1, 1].set_title('Top 10 Cidades por Número de Incidentes')
    else:
        axes[1, 1].text(0.5, 0.5, 'Dados de cidade não disponíveis', 
                        ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Análise por Cidade')
    
    plt.tight_layout()
    plt.show()

else:
    print("Coluna 'countrycode' não encontrada no dataset.")


## 8. Resumo e Conclusões


In [ ]:
# Resumo e conclusões da análise
print("=== RESUMO E CONCLUSÕES DA ANÁLISE ===")

print(f"\n📊 ESTATÍSTICAS GERAIS:")
print(f"  • Total de registros: {len(df):,}")
print(f"  • Total de colunas: {len(df.columns)}")
print(f"  • Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

if 'incidentgrade' in df.columns:
    target_counts = df['incidentgrade'].value_counts()
    print(f"\n VARIÁVEL TARGET (IncidentGrade):")
    print(f"  • FalsePositive: {target_counts.get(0, 0):,} ({target_counts.get(0, 0)/len(df)*100:.1f}%)")
    print(f"  • TruePositive: {target_counts.get(1, 0):,} ({target_counts.get(1, 0)/len(df)*100:.1f}%)")
    print(f"  • BenignPositive: {target_counts.get(2, 0):,} ({target_counts.get(2, 0)/len(df)*100:.1f}%)")

if 'category' in df.columns:
    category_counts = df['category'].value_counts()
    print(f"\n CATEGORIAS DE INCIDENTES:")
    print(f"  • Total de categorias: {len(category_counts)}")
    print(f"  • Categoria mais frequente: Categoria {category_counts.index[0]} ({category_counts.iloc[0]:,} incidentes)")

if 'timestamp' in df.columns:
    df['datetime'] = pd.to_datetime(df['timestamp'], errors='coerce')
    print(f"\n ANÁLISE TEMPORAL:")
    print(f"  • Período: {df['datetime'].min()} a {df['datetime'].max()}")
    print(f"  • Total de dias: {(df['datetime'].max() - df['datetime'].min()).days}")
    
    # Hora com mais incidentes
    hourly_counts = df['hour'].value_counts().sort_index()
    peak_hour = hourly_counts.idxmax()
    print(f"  • Hora de pico: {peak_hour:02d}:00 ({hourly_counts.max():,} incidentes)")

if 'countrycode' in df.columns:
    country_counts = df['countrycode'].value_counts()
    print(f"\n ANÁLISE GEOGRÁFICA:")
    print(f"  • Total de países: {len(country_counts)}")
    print(f"  • País com mais incidentes: País {country_counts.index[0]} ({country_counts.iloc[0]:,} incidentes)")

print(f"\n QUALIDADE DOS DADOS:")
print(f"  • Valores ausentes: {df.isnull().sum().sum()}")
print(f"  • Valores infinitos: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum()}")
print(f"  • Tipos de dados: {df.dtypes.value_counts().to_dict()}")

print(f"\n PRINCIPAIS INSIGHTS:")
print(f"  1. Dataset com alta qualidade após processamento ETL")
print(f"  2. Variável target com distribuição desbalanceada")
print(f"  3. Múltiplas categorias de incidentes identificadas")
print(f"  4. Padrões temporais e geográficos claros")
print(f"  5. Dados prontos para modelagem de Machine Learning")

print(f"\n PRÓXIMOS PASSOS RECOMENDADOS:")
print(f"  1. Balanceamento de classes para melhor performance do modelo")
print(f"  2. Feature engineering para capturar padrões temporais")
print(f"  3. Seleção de features baseada em correlações")
print(f"  4. Aplicação de algoritmos de classificação")
print(f"  5. Validação cruzada e otimização de hiperparâmetros")

print(f"\n APLICAÇÕES POTENCIAIS:")
print(f"  • Sistema de detecção automática de incidentes")
print(f"  • Classificação de ameaças em tempo real")
print(f"  • Análise preditiva de riscos de segurança")
print(f"  • Dashboard de monitoramento de segurança")
print(f"  • Sistema de alertas inteligentes")

print(f"\n{'='*60}")
print(f" ANÁLISE CONCLUÍDA COM SUCESSO!")
print(f"{'='*60}")
